In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *

# 应用架构深度解析

我们现在处于一个令人兴奋的位置，可以完全理解我们一直在用于计算机视觉、自然语言处理和表格分析的最先进模型的架构。在本章中，我们将补充所有关于fastai应用模型如何工作的缺失细节，并向你展示如何构建它们所使用的模型。

我们还将回顾我们在<<chapter_midlevel_data>>中看到的用于孪生网络的自定义数据预处理管道，并展示你如何使用fastai库中的组件为新任务构建自定义预训练模型。

我们将从计算机视觉开始。

## 计算机视觉

对于计算机视觉应用，我们根据任务的不同使用`vision_learner`和`unet_learner`函数来构建我们的模型。在本节中，我们将探讨如何构建本书第一部分和第二部分中我们所使用的`Learner`对象。

### 视觉学习者

让我们看看使用`vision_learner`函数时会发生什么。我们首先传递这个函数一个用于网络*主体*的架构。大多数时候我们使用ResNet，你已经知道如何创建它，所以我们不需要进一步深入。预训练权重会根据需要被下载并加载到ResNet中。

然后，对于迁移学习，网络需要被*剪切*。这指的是切掉最后的层，它只负责ImageNet特定的分类。实际上，我们不仅仅切掉这一层，而是从自适应平均池化层开始的所有层。原因很快就会变得清晰。由于不同的架构可能使用不同类型的池化层，甚至完全不同的*头部*，我们不仅仅寻找自适应池化层来决定在哪里剪切预训练模型。相反，我们有一个信息字典，用于每个模型来确定其主体在哪里结束，头部从哪里开始。我们称这个为`model_meta`——这是resnet-50的：

In [ ]:
model_meta[resnet50]

{'cut': -2,
 'split': <function fastai.vision.learner._resnet_split(m)>,
 'stats': ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])}

术语：主体和头部：神经网络的“头部”是专门针对特定任务的部分。对于CNN来说，通常是自适应平均池化层之后的部分。“主体”是其他所有部分，包括我们在<<chapter_resnet>>中学到的“stem”。

如果我们取所有在剪切点`-2`之前的层，我们将得到fastai用于迁移学习模型的部分。现在，我们添加我们的新头部。这是使用`create_head`函数创建的：

In [ ]:
#hide_output
create_head(20,2)

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): full: False
  (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=20, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=False)
)

```
Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=20, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=False)
)
```

使用这个函数，你可以选择在末端添加多少个额外的线性层，每个线性层之后使用多少dropout，以及使用哪种池化。默认情况下，fastai会同时应用平均池化和最大池化，并将两者连接在一起（这是`AdaptiveConcatPool2d`层）。这不是一个特别常见的方法，但它是在近年来在fastai和其他研究实验室独立开发的，并且往往会比仅使用平均池化提供一些微小的改进。

fastai与大多数库有所不同，它默认在CNN头部添加两个线性层，而不是一个。原因是即使在将预训练模型转移到非常不同的领域时，迁移学习仍然可能有用，正如我们所看到的。然而，在这些情况下，仅使用一个线性层可能不够；我们发现使用两个线性层可以使迁移学习在更多情况下更快、更容易地被使用。

> 注意：最后一个Batchnorm？：`create_head`的一个值得关注的参数是`bn_final`。将其设置为`true`将会在最终层添加一个批量归一化层。这在帮助你的模型适当地为你的输出激活进行缩放时非常有用。我们还没有看到这种方法在任何地方发表过，但我们发现在实践中，无论在哪里使用它，它都能很好地工作。

现在让我们看看`unet_learner`在<<chapter_intro>>中展示的分割问题中做了什么。

### unet_learner

在深度学习中最有趣的架构之一就是我们在<<chapter_intro>>中用于分割的架构。分割是一个具有挑战性的任务，因为所需的输出实际上是一个图像，或者是一个像素网格，其中包含了每个像素的预测标签。还有其他一些任务具有类似的基本设计，例如提高图像的分辨率（*超分辨率*）、给黑白图像上色（*彩色化*）或将照片转换成合成的绘画（*风格迁移*）——这些任务在本书的[在线](https://book.fast.ai/)章节中有介绍，所以在阅读本章之后一定要查看。在每种情况下，我们都是从一个图像开始，将其转换为具有相同尺寸或宽高比的其他图像，但像素以某种方式被改变。我们将这些称为*生成性视觉模型*。

我们实现这一点的方法是采用与上一问题中看到的开发CNN头部完全相同的方法。我们从ResNet开始，例如，然后切掉自适应池化层及其之后的所有层。然后我们用自定义的头部替换这些层，它执行生成任务。

最后一句话有很多模糊的地方！我们究竟如何创建一个能够生成图像的CNN头部？如果我们从一个224像素的输入图像开始，那么在ResNet主体的末端，我们将得到一个7×7的卷积激活网格。我们如何将这个网格转换成一个224像素的分割掩模？

自然，我们用神经网络来实现这一点！所以我们需要某种能够增加CNN网格大小的层。一种非常简单的方法是用一个2×2正方形中的四个像素替换7×7网格中的每个像素。这四个像素将具有相同的值——这被称为*最近邻插值*。PyTorch为我们提供了一个执行此操作的层，所以一个选择是创建一个头部，其中包含步长为1的卷积层（以及通常的批量归一化和ReLU层），并在它们之间穿插2×2最近邻插值层。实际上，你现在就可以尝试！看看你是否能创建一个像这样设计的自定义头部，并在CamVid分割任务上尝试它。你会发现你会得到一些合理的结果，尽管它们不会像我们的<<chapter_intro>>结果那样好。

另一种方法是用*转置卷积*（也称为*步长为0.5的卷积*）替换最近邻和卷积的组合。这与常规卷积相同，但首先在输入的所有像素之间插入零填充。这最好通过图片来理解——<<transp_conv>>展示了我们在<<chapter_convolutions>>中讨论的优秀[卷积算术论文](https://arxiv.org/abs/1603.07285)中的一个图表，展示了一个3×3转置卷积应用于一个3×3图像。

<img alt="A transposed convolution" width="815" caption="A transposed convolution (courtesy of Vincent Dumoulin and Francesco Visin)" id="transp_conv" src="images/att_00051.png">

正如你所看到的，这些方法的结果都是增加输入的大小。你现在可以尝试使用fastai的`ConvLayer`类；通过传递参数`transpose=True`来创建一个转置卷积，而不是常规的卷积，在你的自定义头部。

然而，这两种方法都不是特别有效。问题是我们7×7的网格没有足够的信息来创建一个224×224像素的输出。这要求每个网格单元的激活包含足够的信息来完全重建输出中的每个像素。解决这个问题的方法是使用*跳跃连接*，就像在ResNet中一样，但是跳过ResNet主体的激活，直接跳到架构另一侧的转置卷积的激活。这种方法在<<unet>>中有所展示，是由Olaf Ronneberger、Philipp Fischer和Thomas Brox在2015年的论文["U-Net: Convolutional Networks for Biomedical Image Segmentation"](https://arxiv.org/abs/1505.04597)中提出的。尽管这篇论文专注于医学应用，但U-Net已经彻底改变了所有类型的生成性视觉模型。

<img alt="The U-Net architecture" width="630" caption="The U-Net architecture (courtesy of Olaf Ronneberger, Philipp Fischer, and Thomas Brox)" id="unet" src="images/att_00052.png">

这张图片展示了左边的CNN主体（在本例中，它是一个普通的CNN，不是ResNet，并且它们使用的是2×2最大池化而不是步长为2的卷积，因为这篇论文是在ResNets出现之前写的），右边是转置的卷积层（“上-卷积”）。然后额外的跳跃连接以灰色箭头的形式从左到右显示（这些有时被称为*交叉连接*）。你可以看到为什么它被称为“U-Net”。

有了这种架构，转置卷积的输入不仅仅是前一层的低分辨率网格，还有ResNet头部的高分辨率网格。这允许U-Net使用原始图像的所有信息，因为这是必需的。U-Net的一个挑战是确切的架构取决于图像的大小。fastai有一个独特的`DynamicUnet`类，它根据提供的数据自动生成合适大小的架构。

现在让我们专注于一个例子，我们将利用fastai库来编写一个自定义模型。

### 孪生网络

In [ ]:
#hide
from fastai.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

class SiameseImage(fastuple):
    def show(self, ctx=None, **kwargs): 
        img1,img2,same_breed = self
        if not isinstance(img1, Tensor):
            if img2.size != img1.size: img2 = img2.resize(img1.size)
            t1,t2 = tensor(img1),tensor(img2)
            t1,t2 = t1.permute(2,0,1),t2.permute(2,0,1)
        else: t1,t2 = img1,img2
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), 
                          title=same_breed, ctx=ctx)
    
def label_func(fname):
    return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

class SiameseTransform(Transform):
    def __init__(self, files, label_func, splits):
        self.labels = files.map(label_func).unique()
        self.lbl2files = {l: L(f for f in files if label_func(f) == l) for l in self.labels}
        self.label_func = label_func
        self.valid = {f: self._draw(f) for f in files[splits[1]]}
        
    def encodes(self, f):
        f2,t = self.valid.get(f, self._draw(f))
        img1,img2 = PILImage.create(f),PILImage.create(f2)
        return SiameseImage(img1, img2, t)
    
    def _draw(self, f):
        same = random.random() < 0.5
        cls = self.label_func(f)
        if not same: cls = random.choice(L(l for l in self.labels if l != cls)) 
        return random.choice(self.lbl2files[cls]),same
    
splits = RandomSplitter()(files)
tfm = SiameseTransform(files, label_func, splits)
tls = TfmdLists(files, tfm, splits=splits)
dls = tls.dataloaders(after_item=[Resize(224), ToTensor], 
    after_batch=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)])

让我们回顾一下在<<chapter_midlevel_data>>中为孪生网络设置的输入管道。如果你还记得，它由一对图像组成，标签是`True`或`False`，这取决于它们是否属于同一类别。

利用我们刚刚看到的内容，让我们为这个任务构建一个定制模型并训练它。怎么做呢？我们将使用预训练的架构，并将我们的两幅图像通过它传递。然后我们可以连接结果，并将它们发送到一个定制的头部，它将返回两个预测。在模块方面，这看起来像这样：

In [ ]:
class SiameseModel(Module):
    def __init__(self, encoder, head):
        self.encoder,self.head = encoder,head
    
    def forward(self, x1, x2):
        ftrs = torch.cat([self.encoder(x1), self.encoder(x2)], dim=1)
        return self.head(ftrs)

为了创建我们的编码器，我们只需要取一个预训练的模型并进行裁剪，就像我们之前解释的那样。`create_body`函数为我们完成了这项工作；我们只需要传递给它我们想要裁剪的位置。正如我们之前看到的，根据预训练模型的元数据字典，ResNet的裁剪值是`-2`：

In [ ]:
encoder = create_body(resnet34, cut=-2)

接下来我们可以创建我们的头部（head）。查看编码器后我们知道最后一层有512个特征，所以这个头部需要接收`512*2`个特征。为什么是2？我们需要乘以2是因为我们有两幅图像。所以我们按照以下方式创建头部：

In [ ]:
head = create_head(512*2, 2, ps=0.5)

有了我们的编码器和头部，我们现在可以构建我们的模型了：

In [ ]:
model = SiameseModel(encoder, head)

在使用`Learner`之前，我们还有两件事情需要定义。首先，我们必须定义我们想要使用的损失函数。它是常规的交叉熵，但由于我们的目标是布尔值，我们需要将它们转换为整数，否则PyTorch会抛出错误：

In [ ]:
def loss_func(out, targ):
    return nn.CrossEntropyLoss()(out, targ.long())

更重要的是，为了充分利用迁移学习，我们必须定义一个自定义的*分割器*（splitter）。分割器是一个函数，它告诉fastai库如何将模型分割成参数组。这些在后台用于在进行迁移学习时仅训练模型的头部。

在这里，我们想要两个参数组：一个用于编码器，一个用于头部。因此，我们可以定义以下分割器（`params`只是一个返回给定模块所有参数的函数）：

In [ ]:
def siamese_splitter(model):
    return [params(model.encoder), params(model.head)]

然后我们可以通过传递数据、模型、损失函数、分割器以及我们想要的任何指标来定义我们的`Learner`。由于我们没有使用fastai的迁移学习便捷函数（如`vision_learner`），我们必须手动调用`learn.freeze`。这将确保只有最后一个参数组（在本例中是头部）被训练：

In [ ]:
learn = Learner(dls, model, loss_func=loss_func, 
                splitter=siamese_splitter, metrics=accuracy)
learn.freeze()

然后我们可以直接使用通常的方法来训练我们的模型：

In [ ]:
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.367015,0.281242,0.885656,00:26
1,0.307688,0.214721,0.915426,00:26
2,0.275221,0.170615,0.936401,00:26
3,0.223771,0.159633,0.943843,00:26


在解冻并使用有区别的学习率进一步微调整个模型之前（也就是说：对于主体使用较低的学习率，对于头部使用较高的学习率）：

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, slice(1e-6,1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.212744,0.159033,0.944520,00:35
1,0.201893,0.159615,0.942490,00:35
2,0.204606,0.152338,0.945196,00:36
3,0.213203,0.148346,0.947903,00:36


当我们记得一个以相同方式训练的分类器（没有数据增强）的错误率是7%时，94.8%的表现是非常好的。

既然我们已经了解了如何创建最先进的计算机视觉模型，现在让我们转向自然语言处理（NLP）。

## 自然语言处理

将AWD-LSTM语言模型转换为迁移学习分类器的过程，与我们在本章第一节中使用`vision_learner`的过程非常相似。在这种情况下，我们不需要“meta”字典，因为我们不需要支持如此多样的架构。我们所要做的就是选择语言模型中的堆叠RNN作为编码器，它是一个单一的PyTorch模块。这个编码器将为输入的每个单词提供一个激活值，因为语言模型需要为每个下一个单词输出一个预测。

为了从这个模型创建一个分类器，我们采用了[ULMFiT论文](https://arxiv.org/abs/1801.06146)中描述的“BPTT for Text Classification (BPT3C)”方法：

我们将文档分成固定长度的批次，大小为*b*。在每个批次的开始，模型会用前一个批次的最终状态进行初始化；我们跟踪用于平均池化和最大池化的隐藏状态；梯度会反向传播到那些对最终预测有贡献的隐藏状态的批次。在实际操作中，我们使用可变长度的反向传播序列。

换句话说，分类器包含一个`for`循环，该循环遍历序列的每个批次。状态在批次之间保持不变，并且存储每个批次的激活值。最后，我们使用与计算机视觉模型相同的平均和最大池化技巧——但这一次，我们不是在CNN网格单元上进行池化，而是在RNN序列上进行池化。

为了这个`for`循环，我们需要将数据分批处理，但每段文本都需要单独处理，因为它们各自有自己的标签。然而，很可能这些文本的长度并不相同，这意味着我们不能像处理语言模型那样将它们全部放入同一个数组。

这就是填充（padding）发挥作用的地方：在抓取一组文本时，我们确定长度最长的一个，然后用一个特殊的标记`xxpad`填充那些较短的文本。为了避免极端情况，比如在同一批次中有一段文本有2000个标记，而另一段只有10个标记（这样会有很多填充，以及大量浪费的计算），我们通过确保长度相近的文本被放在一起来改变随机性。文本在训练集上仍然会以某种随机顺序排列（对于验证集，我们可以简单地按长度顺序排序），但不会完全随机。

当创建我们的`DataLoaders`时，fastai库会自动在后台完成这些操作。

## Tabular

最后，让我们看看`fastai.tabular`模型。（我们不需要单独查看协同过滤，因为我们已经看到这些模型只是表格模型，或者使用我们在前面从头实现的点积方法。）

这是`TabularModel`的`forward`方法：

```python
if self.n_emb != 0:
    x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
    x = torch.cat(x, 1)
    x = self.emb_drop(x)
if self.n_cont != 0:
    x_cont = self.bn_cont(x_cont)
    x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
return self.layers(x)
```

我们在这里不会展示`__init__`，因为它没有那么有趣，但我们会逐行查看`forward`中的代码。第一行：

```python
if self.n_emb != 0:
```

这行代码只是检查是否有任何嵌入（embeddings）需要处理——如果我们只有连续变量，我们可以跳过这部分。`self.embeds`包含了嵌入矩阵，所以这行代码获取每个嵌入的激活值：

```python
    x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
```

然后将它们连接成一个单一的张量：

```python
    x = torch.cat(x, 1)
```

然后应用dropout。你可以通过传递`embd_p`到`__init__`来改变这个值：

```python
    x = self.emb_drop(x)
```

现在我们检查是否有任何连续变量需要处理：

```python
if self.n_cont != 0:
```

它们通过一个批量归一化层：

```python
    x_cont = self.bn_cont(x_cont)
```

然后与嵌入激活值连接，如果有的话：

```python
    x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
```

最后，这通过线性层（如果`use_bn`为`True`，每个都包括批量归一化，如果`ps`设置为某个值或值的列表，则包括dropout）：

```python
return self.layers(x)
```

恭喜你！现在你知道了fastai库中使用的所有架构的每一个部分！

## 架构总结

正如你所看到的，深度学习架构的细节现在不应该让你感到害怕。你可以查看fastai和PyTorch的代码，看看究竟发生了什么。更重要的是，尝试理解为什么会这样。看看代码中引用的论文，试着看看代码是如何与描述的算法相匹配的。

现在我们已经调查了模型的所有部分以及输入数据，我们可以思考这对实际深度学习意味着什么。如果你有无限的数据、无限的内存和无限的时间，那么建议就很简单了：在你的所有数据上训练一个巨大的模型，训练很长时间。但深度学习的复杂性在于，你的数据、内存和时间通常是有限。如果你的内存或时间不足，那么解决方案就是训练一个较小的模型。如果你无法训练足够长的时间以过拟合，那么你就没有充分利用你模型的容量。

所以，第一步是达到可以过拟合的程度。然后问题是如何减少这种过拟合。<<reduce_overfit>>展示了我们推荐从那里开始优先考虑的步骤。

<img alt="Steps to reducing overfitting" width="400" caption="Steps to reducing overfitting" id="reduce_overfit" src="images/att_00047.png">

许多实践者在面对过拟合的模型时，会从完全错误的地方开始。他们的起点是使用一个更小的模型，或者更多的正则化。除非你训练模型所需的时间或内存过多，否则减小模型的大小应该是你最后采取的步骤。减小模型的大小会降低模型学习数据中微妙关系的能力。

相反，你的第一步应该是寻求*创造更多数据*。这可能涉及为你已经拥有的数据添加更多标签，寻找你的模型可以解决的额外任务（或者换句话说，识别你可以建模的不同种类的标签），或者通过使用更多或不同的数据增强技术创建额外的合成数据。得益于Mixup和类似方法的发展，现在几乎所有类型的数据都可以进行有效的数据增强。

一旦你获得了尽可能多的数据，并通过利用所有能找到的标签和进行所有有意义的增强来尽可能有效地使用它，如果你仍然过拟合，那么你应该考虑使用更具有泛化能力的架构。例如，添加批量归一化可能会提高泛化能力。

如果你在使用数据和调整架构方面已经尽力了，但仍然过拟合，那么你可以看看正则化。一般来说，在最后一两层添加dropout可以很好地正则化你的模型。然而，正如我们从AWD-LSTM的发展故事中学到的，通常在模型的不同部分添加不同类型的dropout可以帮助更多。一般来说，一个更大的模型具有更多的正则化，因此可以比一个更小、正则化较少的模型更灵活，也更准确。

只有在考虑了所有这些选项之后，我们才会建议你尝试使用你架构的更小版本。

## 问卷调查

1. 神经网络的“头部”是什么？
2. 神经网络的“身体”是什么？
3. “切割”神经网络是什么意思？为什么在迁移学习中我们需要这样做？
4. `model_meta`是什么？尝试打印出来看看里面有什么。
5. 阅读`create_head`的源代码，确保你理解每一行的作用。
6. 查看`create_head`的输出，确保你理解为什么每一层存在，以及`create_head`源代码是如何创建它们的。
7. 找出如何改变`vision_learner`创建的dropout、层大小和层数，并看看你是否能找出能提高宠物识别器准确性的值。
8. `AdaptiveConcatPool2d`是做什么的？
9. “最近邻插值”是什么？它如何用于上采样卷积激活？
10. “转置卷积”是什么？它还有什么别的名字？
11. 创建一个`transpose=True`的卷积层，将其应用于图像。检查输出形状。
12. 画出U-Net架构。
13. “BPTT for Text Classification”（BPT3C）是什么？
14. 在BPT3C中我们如何处理不同长度的序列？
15. 尝试在一个笔记本中，逐行运行`TabularModel.forward`的每一行，每行一个单元格，并查看每一步的输入和输出形状。
16. `TabularModel`中的`self.layers`是如何定义的？
17. 防止过拟合的五个步骤是什么？
18. 为什么我们在尝试其他防止过拟合的方法之前不降低架构复杂度？

### 进一步研究

1. 编写你自己的自定义头部，并尝试用它来训练宠物识别器。看看你是否能比fastai的默认设置获得更好的结果。
2. 在CNN头部中尝试切换`AdaptiveConcatPool2d`和`AdaptiveAvgPool2d`，看看它们之间有什么不同。
3. 编写你自己的自定义分割器，为每个ResNet块创建一个独立的参数组，以及为stem创建一个独立的组。尝试用它进行训练，看看是否能提高宠物识别器的性能。
4. 阅读有关生成图像模型的在线章节，并创建你自己的颜色化模型、超分辨率模型或风格迁移模型。
5. 使用最近邻插值创建一个自定义头部，并用它在CamVid上进行分割。